## The Model

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
#%pip install autogen-ext[openai]
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo, UserMessage
model_client = OpenAIChatCompletionClient(model="xiaomi/mimo-v2-flash:free", base_url="https://openrouter.ai/api/v1",
                                          model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown", structured_output=False))
#this model_info is required for OpenRouter models only, for OpenAI there is no need to pass the model_info

result = await model_client.create([UserMessage(content="hello World", source="user")])
print(result)

finish_reason='stop' content='Hello! How can I help you today?' usage=RequestUsage(prompt_tokens=39, completion_tokens=10) cached=False logprobs=None thought=None


## Message

In [81]:
#%pip install autogen-agentchat
from autogen_agentchat.messages import TextMessage
message = TextMessage(content="Hello, do you have any information from the tool", source="user")

## Agents

In [73]:
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken

agent = AssistantAgent(
    name="first_agent",
    model_client=model_client,
    system_message="You are a stand up comedian.",
    model_client_stream=True)

response = await agent.on_messages([message], cancellation_token=CancellationToken())
print(response.chat_message.content)

My agent says I'm not getting peanuts for this gig, so I guess I'm playing for my health.  And let me tell you, my health isn't what it used to be!


In [75]:
def get_trending_topics() -> list:
    """ Information required by the agent """
    return ["husband wife"]

In [87]:
## Agent with tools
agent_withTools = AssistantAgent(
    name="agent_with_tools",
    model_client=model_client,
    system_message="Print whatever the tool returns to you.",
    tools = [get_trending_topics],
    model_client_stream=True,
    reflect_on_tool_use=True)

response = await agent_withTools.on_messages([message], cancellation_token=CancellationToken())
print(response.chat_message.content)

I see the trending topic is **"husband wife"**. Would you like me to generate some content ideas related to this theme?


In [83]:
for inner_message in response.inner_messages:
    print(inner_message.content)

I don't have any information from the tool yet. To get the trending topics, I need to make a call to the `get_trending_topics` function. Let me do that for you.
[FunctionCall(id='call_37a2bb7b6cdb4cb29eec2280', arguments='{}', name='get_trending_topics')]
[FunctionExecutionResult(content="['husband wife']", name='get_trending_topics', call_id='call_37a2bb7b6cdb4cb29eec2280', is_error=False)]


## Teams

In [88]:
## Teams
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

text_termination  = TextMentionTermination("Stop")
team = RoundRobinGroupChat([agent, agent_withTools], termination_condition=text_termination, max_turns=20)

response = await team.run(task="go")

for message in response.messages:
    print(message)

Error processing publish message for first_agent_64c4b301-25ad-472b-b525-fab1101323e9/64c4b301-25ad-472b-b525-fab1101323e9
Traceback (most recent call last):
  File "C:\Users\vishi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vishi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vishi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    r

RuntimeError: RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1767398400000'}, 'provider_name': None}}, 'user_id': 'user_37WulDpYYUHkMI0NNaoEX94OKj2'}
Traceback:
Traceback (most recent call last):

  File "C:\Users\vishi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "C:\Users\vishi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 953, in on_messages_stream
    async 